In [20]:
import pandas as pd
import geopandas as gpd
import requests
import geojson
from pyproj import CRS
from shapely.geometry import Point

def get_gazetteer():
    """
    Retrieve GeoNames gazetteer and limit that to contain only the toponyms in the Jyväskylä area.
    """
    #retrieve the toponyms from Geonames txt file
    geonames = pd.read_csv(r"FI.txt", sep="\t", header= None)
    #rename the columns
    geonames.columns = ["geonameid", "name", "asciiname", "alternatenames", "latitude", "longitude", "feature class", 
                        "feature code", "country code", "cc2", "admin1 code", "admin2 code", "admin3 code", "admin4 code", "population", "elevation", "dem", "timezone", "modification date"]
      
    #make shapely points from latitude and longitude and convert to geodataframe
    geonames["geometry"] = geonames.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)
    geonames = gpd.GeoDataFrame(geonames)

    #reproject geonames to ETRS89 / TM35FIN
    geonames.crs = CRS.from_epsg(4326).to_wkt()
    geonames = geonames.to_crs(epsg=3067)
    
    
    
    # Fetch city border data from WFS using requests
    r = requests.get('http://geo.stat.fi/geoserver/tilastointialueet/wfs', params=dict(service='WFS', version='2.0.0', request='GetFeature', typeName='tilastointialueet:kunta1000k', outputFormat='json'))

    # get a shapefile of municipalities
    municip = gpd.GeoDataFrame.from_features(geojson.loads(r.content),  crs="EPSG:3067")
    
    # only keep Jyväskylä area
    jyv = municip.loc[municip["nimi"] == "Jyväskylä"]
    jyv = jyv.to_crs(epsg=3067)
    
    #retrieve the placenames in Jyväskylä
    jyvnames = gpd.overlay(geonames, jyv, how="intersection")
    jyvnames.crs = CRS.from_epsg(3067)
    
    #convert all placenames to small letters for further processing
    jyvnames["name"] = jyvnames["name_1"].str.lower()
    
    #delete general city name "Jyväskylä" for ambigous results (done in post-processing)
    jyvnames = jyvnames[(jyvnames.name != "jyväskylä")]
    jyvnames['bbox'] = jyvnames['bbox'].astype(str)  
    print("Gazetteer ready for use")
    return jyvnames


df = get_gazetteer()
df['bbox_s'] = df['bbox'].astype(str)  
df.to_file("jyv_gazetteer.gpkg", driver='GPKG')


C:\Hyapp\Anaconda3\envs\ylli\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Gazetteer ready for use
